some of the code in this lab was copied from the solution at https://github.com/learn-co-curriculum/dsc-sql-subqueries-lab/tree/solution

# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [1]:
#Your code here; import the necessary packages
import pandas as pd
import sqlite3

In [2]:
#Your code here; create the connection and cursor
conn = sqlite3.connect('data.sqlite')
c = conn.cursor()

## Write an Equivalent Query using a Subquery

```SQL
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';
```

In [4]:
#Your code here; use a subquery. No join will be necessary.
c.execute(''' SELECT customerNumber, contactLastName, contactFirstName
                FROM customers 
                WHERE customerNumber IN (SELECT customerNumber FROM orders WHERE orderDate = 
                                            '2003-01-31') 
                                            ''')
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head(7)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [5]:
#Your code here
c.execute("""SELECT productName, COUNT(orderNumber) as numberOrders, SUM(quantityOrdered) as totalUnitsSold
               FROM products
               JOIN orderdetails
               USING (productCode)
               GROUP BY 1
               ORDER BY totalUnitsSold desc;
               """)


df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,productName,numberOrders,totalUnitsSold
0,1992 Ferrari 360 Spider red,53,1808
1,1937 Lincoln Berline,28,1111
2,American Airlines: MD-11S,28,1085
3,1941 Chevrolet Special Deluxe Cabriolet,28,1076
4,1930 Buick Marquette Phaeton,28,1074


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

In [8]:
#Your code here
c.execute("""SELECT productName, COUNT(DISTINCT customerNumber) AS numPurchasers
               FROM products
               JOIN orderdetails
               USING(productCode)
               JOIN orders
               USING(orderNumber)
               GROUP BY 1
               ORDER BY numPurchasers DESC;
               """)

df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]

df

,productName,numPurchasers
0,1992 Ferrari 360 Spider red,40
1,1934 Ford V8 Coupe,27
2,1952 Alpine Renault 1300,27
3,1972 Alfa Romeo GTA,27
4,Boeing X-32A JSF,27
5,18th Century Vintage Horse Carriage,26
6,1900s Vintage Bi-Plane,26
7,1913 Ford Model T Speedster,26
8,1937 Horch 930V Limousine,26
9,1940 Ford Delivery Sedan,26


## Select the Employee Number, Office Code, City (of the office), and Name (First and Last) of those Employees who Sold Products that Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [10]:
#Your code here
c.execute("""SELECT DISTINCT employeeNumber, officeCode, o.city, firstName, lastName
               FROM employees e
               JOIN offices o
               USING(officeCode)
               JOIN customers c
               ON e.employeeNumber = c.salesRepEmployeeNumber
               JOIN orders
               USING(customerNumber)
               JOIN orderdetails
               USING(orderNumber)
               WHERE productCode IN (SELECT productCode
                                            FROM products
                                            JOIN orderdetails
                                            USING(productCode)
                                            JOIN orders
                                            USING(orderNumber)
                                            GROUP BY productCode
                                            HAVING COUNT(DISTINCT customerNumber) < 20);
                                            """)
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]

df

,employeeNumber,officeCode,city,firstName,lastName
0,1370,4,Paris,Gerard,Hernandez
1,1501,7,London,Larry,Bott
2,1337,4,Paris,Loui,Bondur
3,1166,1,San Francisco,Leslie,Thompson
4,1286,3,NYC,Foon Yue,Tseng
5,1612,6,Sydney,Peter,Marsh
6,1611,6,Sydney,Andy,Fixter
7,1401,4,Paris,Pamela,Castillo
8,1621,5,Tokyo,Mami,Nishi
9,1323,3,NYC,George,Vanauf


## Select the Employee Number, Name (First and Last) and Number of Customers of Employees Who's Customers Have an Average Credit Limit of Over 15K

In [ ]:
SELECT employeeNumber, firstName, lastName, COUNT(salesRepEmployeeNumber)

In [11]:
#Your code here
c.execute("""SELECT employeeNumber, firstName, lastName, COUNT(customerNumber) AS numCustomers
               FROM employees e
               JOIN customers c
               ON e.employeeNumber = c.salesRepEmployeeNumber
               GROUP BY 1,2,3
               HAVING AVG(creditLimit) > 15000;
               """)
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,employeeNumber,firstName,lastName,numCustomers
0,1165,Leslie,Jennings,6
1,1166,Leslie,Thompson,6
2,1188,Julie,Firrelli,6
3,1216,Steve,Patterson,6
4,1286,Foon Yue,Tseng,7
5,1323,George,Vanauf,8
6,1337,Loui,Bondur,6
7,1370,Gerard,Hernandez,7
8,1401,Pamela,Castillo,10
9,1501,Larry,Bott,8


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!